In [0]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd

from google.colab import files
from matplotlib import pyplot
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot


import tensorflow as tf
from tensorflow import keras
import pandas as pd

from google.colab import files
from matplotlib import pyplot
uploaded = files.upload()

import io
'''
#df2 = pd.read_csv(io.BytesIO(uploaded['pls.csv']))
df3 = pd.read_csv(io.BytesIO(uploaded['nasdaq1.csv']))
df3 = df3.iloc[::-1]
#df3 = df3.drop(df3.columns[4])

print(df3)
df3.iloc[:,1] = df3.iloc[:,1].str.replace('$', '')
print(df3)
'''

def parser(x):
	return datetime.strptime(x, '%m/%d/%y')
series = pd.read_csv('nasdaq1.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
print(series)

series = series[::-1]
seriesdf = pd.DataFrame(series)
seriesdf.iloc[:,0] = seriesdf.iloc[:,0].str.replace('$','')
seriesdf = seriesdf.drop(seriesdf.columns[4],axis = 1)
seriesdf = seriesdf.drop(seriesdf.columns[3],axis = 1)
seriesdf = seriesdf.drop(seriesdf.columns[2],axis = 1)
seriesdf = seriesdf.drop(seriesdf.columns[1],axis = 1)

print(seriesdf)
seriesdf.iloc[:,0] = pd.to_numeric(seriesdf.iloc[:,0])
seriesdf.plot()
pyplot.title("AAPL Closing Price 5/2010 - 5/2020")

pyplot.xlabel("Year")
pyplot.ylabel("Closing Price (USD $)")
pyplot.show()

#seriesdf.to_csv('seriesdf.csv')
from google.colab import files
seriesdf.to_csv('seriesdf.csv') 
#files.download('seriesdf.csv')

In [0]:
def timeseries_to_supervised(df, lag=1):
	dfdf = pd.DataFrame(df)
	columns = [dfdf.shift(i) for i in range(1, lag+1)]
	columns.append(dfdf)
	dfdf = pd.concat(columns, axis=1)
	dfdf.fillna(0, inplace=True)
	return dfdf

print(timeseries_to_supervised(seriesdf))
x = timeseries_to_supervised(seriesdf)
# create a differenced series
'''def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return diff'''

def my_diff(data,interval = 1):
  diff = []
  for i in range(interval, len(data.iloc[:,0])):
    value = data.iloc[i,0] - (data.iloc[i - interval,0])
    diff.append(value)
  return diff
print(my_diff(seriesdf))
#x = my_diff(seriesdf)
xdf = pd.DataFrame(x)
xdf.to_csv('xdfcsv.csv')
print(xdf)
#files.download('xdfcsv.csv')


In [0]:
from sklearn import preprocessing

X = seriesdf.values
X = X.reshape(len(X),1)
scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
scaler = scaler.fit(X)
scaled_X = scaler.transform(X)
scaled_series = pd.Series(scaled_X[:,0])
print(len(scaled_X))
print(len(scaled_series))
print(scaled_series)

inverted_X = scaler.inverse_transform(scaled_X)
inverted_series = pd.Series(inverted_X[:,0])

def scale(train, test):
	# fit scaler
	scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled


In [0]:
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from math import sqrt
from matplotlib import pyplot
import numpy


raw = seriesdf.values
diffval = my_diff(seriesdf)
diffval
supervised = timeseries_to_supervised(diffval)
supervisedval = supervised.values
train, test = supervisedval[0:-503],supervisedval[-503:]
scaler,train_scaled,test_scaled = scale(train,test)
print(train)
print(len(test))

# make a one-step forecast

def fit_lstm(train, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True, return_sequences=True))
	model.add(Dense(1,activation='tanh'))
	model.add(Dropout(.5))
	model.add(LSTM(30, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True, return_sequences=True))
	model.add(Dense(1, activation='tanh'))
	model.add(Dropout(.3))
	model.add(LSTM(3, batch_input_shape=(batch_size,X.shape[1],X.shape[2]),stateful=True))
	model.add(Dropout(.2))
	model.add(Dense(1, activation='tanh'))
	model.compile(loss='mean_squared_error', optimizer='adam')
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
		model.reset_states()
	return model


def forecast_lstm(model, batch_size, X):
	X = X.reshape(1, 1, len(X))
	yhat = model.predict(X, batch_size=batch_size)
	return yhat[0,0]

def invert_scale(scaler, X, value):
	new_row = [x for x in X] + [value]
	array = numpy.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]


def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]
lstm_model = fit_lstm(train_scaled, 1, 2, 100)
print(lstm_model)

train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
lstm_model.predict(train_reshaped, batch_size=1)

predictions = list()
for i in range(len(test_scaled)):
	# make one-step forecast
	X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
	yhat = forecast_lstm(lstm_model, 2, X)
	# invert scaling
	yhat = invert_scale(scaler, X, yhat)
	# invert differencing
	yhat = inverse_difference(raw, yhat, len(test_scaled)+1-i)
	# store forecast
	predictions.append(yhat)
	expected = raw[len(train) + i + 1]
	#print('Predicted=%f, Expected=%f' % (yhat, expected))

# report performance
rmse = sqrt(mean_squared_error(raw[-503:], predictions))
print('Test RMSE: %.3f' % rmse)
# line plot of observed vs predicted
pyplot.plot(raw[-503:])
pyplot.plot(predictions)

pyplot.show()

print(test)
print(yhat)
train, test = supervisedval[0:-503],supervisedval[-503:]

#print(seriesdf[-2014:])
#pyplot.plot(seriesdf[-2014:])
#pyplot.plot(predictions)

lastyearpredict = predictions[:-503]
print(len(lastyearpredict))
#lastyeartest = raw[248:503]
pyplot.plot(predictions[-25:])
pyplot.plot(raw[-25:], color = 'red')
pyplot.show()